# Corrective RAG as a Service

In this guide we show you how to use llama-agents to build CRAG (Corrective RAG) by Yan et al. as a service.

In [ ]:
!pip install llama-index tavily-python llama-index-tools-tavily-research

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

## Setup Data, Indexes, and Tools

We do the following:
- Download the Llama2 paper as an example document to build RAG over. **NOTE**: We use LlamaParse to parse the document which requires a [LlamaCloud account](https://cloud.llamaindex.ai/). If you choose to use a purely open-source reader, you can do that too.
- Setup a vector index over this paper
- Setup a web search tool (powered by Tavily)

In [ ]:
!mkdir -p 'data/'
!curl 'https://arxiv.org/pdf/2307.09288.pdf' -o 'data/llama2.pdf'

In [ ]:
# OPTION: use LlamaParse
from llama_parse import LlamaParse

parser = LlamaParse(result_type="text")
docs = parser.load_data("data/llama2.pdf")

# # OPTION: use SimpleDirectoryReader (uses open-source PyPDF)
# from llama_index.core import SimpleDirectoryReader

# reader = SimpleDirectoryReader(input_files=["data/llama2.pdf"])
# docs = reader.load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(docs)
retriever = index.as_retriever()

In [ ]:
# Setup Tavily web search
from llama_index.tools.tavily_research.base import TavilyToolSpec
tavily_tool = TavilyToolSpec(api_key=tavily_ai_apikey)

In [2]:
# Setup LLM
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o")

## Define Agent Services

Here we define three services:
- Another initial RAG service that will return retrieved nodes, as well as how relevant they are to the question.
- A separate web search service that is triggered if there are any irrelevant nodes. Will perform query transformation and web search.
- A final summarization service that takes in a set of documents and returns a final result.

In [ ]:
from llama_agents import ComponentService, ServiceComponent, SimpleMessageQueue

message_queue = SimpleMessageQueue()

def to_service_component(component, message_queue, service_name, description):
    server = ComponentService(
        component=component,
        message_queue=message_queue,
        description=description,
        service_name=service_name,
    )
    service_component = ServiceComponent.from_component_service(server)
    return service_component, server

#### Setup Initial RAG Service

Runs retrieval and relevancy check on retrieved nodes.

In [1]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_pipeline import QueryPipeline

relevancy_prompt_tmpl = PromptTemplate(
    template="""As a grader, your task is to evaluate the relevance of a document retrieved in response to a user's question.

    Retrieved Document:
    -------------------
    {context_str}

    User Question:
    --------------
    {query_str}

    Evaluation Criteria:
    - Consider whether the document contains keywords or topics related to the user's question.
    - The evaluation should not be overly stringent; the primary objective is to identify and filter out clearly irrelevant retrievals.

    Decision:
    - Assign a binary score to indicate the document's relevance.
    - Use 'yes' if the document is relevant to the question, or 'no' if it is not.

    Please provide your binary score ('yes' or 'no') below to indicate the document's relevance to the user question."""
)
relevancy_qp = QueryPipeline(chain=[relevancy_prompt_tmpl, llm])

In [ ]:
# define RAG agent
from llama_index.core.query_pipeline import FnComponent
from typing import Dict

def run_retrieval(input_str: str) -> Dict:
    """Run Retrieval."""
    # retrieves a set of nodes
    retrieved_nodes = retriever.retrieve(input_str)

    # runs a relevancy check
    relevancy_results = []
    for node in retrieved_nodes:
        relevancy = relevancy_qp.run(
            context_str=node.text, query_str=query_str
        )
        relevancy_results.append(relevancy.message.content.lower().strip())
    contains_irrelevant = "no" in relevancy_results

    # get relevant texts
    relevant_texts = [
        retrieved_nodes[i].text
        for i, result in enumerate(relevancy_results)
        if result == "yes"
    ]
    relevant_text = "\n".join(relevant_texts)

    # returns a dictionary of items
    return {"relevant_text": relevant_text, "contains_irrelevant": contains_irrelevant, "input_str": input_str}

retrieval_component = FnComponent(fn=run_retrieval)
retrieval_component_s, retrieval_server = to_service_component(
    retrieval_component, message_queue, "Runs a retrieval + relevancy check", "retrieval_service"
)

#### Setup Web Search Service

Wrap Tavily into a component that performs query transformation and web search.

In [ ]:
from llama_index.core.prompts import PromptTemplate

query_transform_tmpl = PromptTemplate(
    template="""Your task is to refine a query to ensure it is highly effective for retrieving relevant search results. \n
    Analyze the given input to grasp the core semantic intent or meaning. \n
    Original Query:
    \n ------- \n
    {query_str}
    \n ------- \n
    Your goal is to rephrase or enhance this query to improve its search performance. Ensure the revised query is concise and directly aligned with the intended search objective. \n
    Respond with the optimized query only:"""
)
query_transform_qp = QueryPipeline(chain=[query_transform_tmpl, llm])

In [ ]:
def run_web_search(input_str: str) -> str:
    """Run Web Search."""
    
    transformed_query_str = self.transform_query_pipeline.run(
        query_str=input_str
    ).message.content
    # Conduct a search with the transformed query string and collect the results.
    search_results = tavily_tool.search(query_str, max_results=5)
    return "\n".join([result.text for result in search_results])

    
web_search_component = FnComponent(fn=run_web_search)
web_search_component_s, web_server = to_service_component(
    web_search_component, message_queue, "Runs web search", "web_search_service"
)

### Setup Summarization Service

At the end, setup a summarization service that can take in relevant information and join it.

In [ ]:
from llama_index.core.base.response.schema import Response
from typing import Optional

def run_summarization(retrieved_text: str, search_text: Optional[str] = None) -> Response:
    """Run summarization."""
    # use summary index to perform summarization
    search_text = search_text or ""
    documents = [Document(text=relevant_text + "\n" + search_text)]
    index = SummaryIndex.from_documents(documents)
    query_engine = index.as_query_engine()
    return query_engine.query(query_str)

summary_component = FnComponent(fn=run_summarization)
summary_component_s, summary_server = to_service_component(
    summary_component, message_queue, "Run summarization", "summarization_service"
)

## Launch Agent Services

Now that we've setup the main components, we can orchestrate them via our pipeline orchestrator.

In [ ]:
from llama_agents import (
    AgentService,
    ControlPlaneServer,
    SimpleMessageQueue,
    PipelineOrchestrator,
    ServiceComponent,
    ComponentService,
)
from llama_index.core.query_pipeline import Link, InputComponent

pipeline = QueryPipeline(
    module_dict={
        "input": InputComponent(),
        "retrieval_server": retrieval_component_s,
        "web_server": web_component_s,
        # TODO: clean this interface up 
        "summary_server_no_web": summary_component_s,
        "summary_server_web": summary_component_s
    }
)
pipeline.add_link("input", "retrieval_server")
pipeline.add_link(
    "retrieval_server", "web_search", condition_fn=lambda x: x["contains_irrelevant"], input_fn=lambda x: x["input_str"]
)
# if web search is called
pipeline.add_link(
    "retrieval_server", 
    "summary_server_web", 
    dest_key="retrieved_text", 
    condition_fn=lambda x: x["contains_irrelevant"],
    input_fn=lambda x: x["relevant_text"]
)
pipeline.add_link("web_search", "summary_server_web", dest_key="search_text")

# if web search is not called
pipeline.add_link(
    "retrieval_server", 
    "summary_server_no_web",
    dest_key="retrieved_text",
    condition_fn=lambda x: not x["contains_irrelevant"],
    input_fn=lambda x: x["relevant_text"]
)

pipeline_orchestrator = PipelineOrchestrator(pipeline)

control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=pipeline_orchestrator,
)

In [ ]:
from llama_agents.launchers import LocalLauncher

## Define Launcher
launcher = LocalLauncher(
    [retrieval_server, web_server, summary_server],
    control_plane,
    message_queue,
)

In [ ]:
query_str = "What is the functionality of latest ChatGPT memory."
result = launcher.launch_single(query_str)
print(str(result))